In [1]:
import numpy as np
import math
from scipy import signal

In [2]:
r, c = np.ogrid[-3/2: 3 / 2 + 1, -3 / 2: 3 / 2 + 1]

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython import display

In [4]:
from math import *

In [5]:
import numpy as np
import math
from scipy import signal

def gaussian2D(sigma=0.5):
    """
    2D gaussian filter
    """
    size = int(math.ceil(sigma * 6))
    if (size % 2 == 0):
        size += 1
    r, c = np.ogrid[-size / 2: size / 2 + 1, -size / 2: size / 2 + 1]
    g = np.exp(-(c * c + r * r) / (2. * sigma ** 2))
    g = g / (g.sum() + 0.000001)
    
    return g

def box2D(n):
    """
    2D box filter
    """
    box = np.full((n, n), 1. / (n * n))
    return box


def calculate_derivatives(i1, i2, sigma=0.5, n=3):
    """
    Derive Ix, Iy and It in this function

    To derive the spatial derivative in one image, you need to smooth the image with gaussian filter,
    and calculate the derivative, signal.convolve2d and np.gradient might be useful here

    To derive the temporal derivative in two images, you need to filter the images with box filters,
    and then calculate the difference between the results
    """
    box_fil = box2D(n)
    gauss_fil = gaussian2D(sigma)
    
    # take the i1 as the changed image(later happened image)
    #step 1 
    #smooth the i2 image with gaussian filter 
    g_i1 = signal.convolve2d(i1,gauss_fil,mode='same')
    # caculate the gradient of x and y: Ix, Iy
    Ix, Iy = np.gradient(g_i1)
    
    
    #step 2: derive the temporal derivative
    #filter the i1 and i2 using box filters
    b_i1 = signal.convolve2d(i1,box_fil,mode='same')
    b_i2 = signal.convolve2d(i2,box_fil,mode='same')
    
    # the temporal derivatives It
    It = b_i2 - b_i1
    return Ix, Iy, It

In [6]:
def rel_error(x, y):
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# test the calculate_derivatives function
a = np.arange(50, step=2).reshape((5,5))
b = np.roll(a, 1, axis=1)
ix, iy, it = calculate_derivatives(a, b, 3, 3)
correct_ix = np.array([[ 1.19566094,  1.44638748,  1.60119287,  1.62253849,  1.50447539],
                        [ 1.0953402,   1.32258973,  1.4614055,   1.4781469,   1.36814814],
                        [ 0.7722809,   0.92753122,  1.01928721,  1.02535579,  0.94404567],
                        [ 0.25022598,  0.29355951,  0.31471869,  0.30865011,  0.27704506],
                        [-0.04909038, -0.06915144, -0.0875189,  -0.09965607, -0.10218035]])
correct_iy = np.array([[ 0.81434768,  0.67021012,  0.31799052, -0.11348914, -0.33688675],
                       [ 1.06507422,  0.87297609,  0.40606602, -0.16184788, -0.45494985],
                       [ 1.26884674,  1.03627543,  0.47354769, -0.2067465,  -0.55688427],
                       [ 1.37557486,  1.1199824,   0.5038906,  -0.23708941, -0.61757009],
                       [ 1.3555138,   1.10076814,  0.48863828, -0.24442014, -0.62009437]])
correct_it = np.array([[ 1.33333333,  0.88888889, -1.33333333, -1.33333333, -0.88888889],
                       [ 2.,          1.33333333, -2.,         -2.,         -1.33333333],
                       [ 2.,          1.33333333, -2.,         -2.,         -1.33333333],
                       [ 2.,          1.33333333, -2.,         -2.,         -1.33333333],
                       [ 1.33333333,  0.88888889, -1.33333333, -1.33333333, -0.88888889]])

print('Testing derivatives:')
print('Ix difference: ', rel_error(ix, correct_ix))
print('Iy difference: ', rel_error(iy, correct_iy))
print('It difference: ', rel_error(it, correct_it))

Testing derivatives:
Ix difference:  3.7486934875982425e-08
Iy difference:  1.1881858448439447e-08
It difference:  1.2500021866561346e-09
